In [ ]:
!pip install earthengine-api geemap folium pandas matplotlib scikit-learn
! earthengine authenticate


import ee
import geemap
import ipywidgets as widgets
from IPython.display import display, clear_output

E0000 00:00:1751147095.155609   48539 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751147095.170922   48539 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Authenticate: Limited support in Colab. Use ee.Authenticate() or --auth_mode=notebook instead.
Authenticate: Credentials already exist.  Use --force to refresh.


In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

import requests
import datetime
import json

In [ ]:
aoi = ee.Geometry.Polygon([[
    [0.09902525852847788,35.77306854462832],
    [0.103316792952306,35.772615897913916],
    [0.1052694411151478,35.77249403105059],
    [0.10868121098209116,35.77242439275912],
    [0.10818768452335092,35.77541878422457],
    [0.10297347019839975,35.77573214425564],
    [0.10121394108463022,35.775819188489564],
    [0.10048438023257944,35.775801779650415],
    [0.09913254688907358,35.775575464394464]
]])

today = ee.Date(datetime.datetime.utcnow().strftime('%Y-%m-%d'))
start = today.advance(-10, 'day')

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import requests
from datetime import datetime

In [ ]:
# 📅 التاريخ اليوم
today = datetime.utcnow().strftime('%Y-%m-%d')

# 🛰️ أحدث صورة Sentinel-2
image = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(aoi) \
    .filterDate(ee.Date(today).advance(-10, 'day'), ee.Date(today)) \
    .sort('system:time_start', False) \
    .first()

In [ ]:
# 🧮 حساب NDVI و NDWI
def add_indices(img):
    ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = img.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return img.addBands([ndvi, ndwi])

image = add_indices(image)

stats = image.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=polygon,
    scale=10,
    maxPixels=1e9
).getInfo()

ndvi = round(stats.get('NDVI', 0), 3)
ndwi = round(stats.get('NDWI', 0), 3)
date = image.date().format("YYYY-MM-dd").getInfo()

print(f"✅ {date} - NDVI: {ndvi} | NDWI: {ndwi}")

NameError: name 'polygon' is not defined